In [2]:
# FIN9013 Assignment 1
# Part A

import pandas as pd
import numpy as np
import statsmodels.api as sm

data = pd.read_stata('ccm_1985_2020_cln_stata-1.dta')

data = data[(data['fyear'] >= 2005) & (data['fyear'] <= 2016)]

data['Debt'] = (data['dltt'] + data['dlc']) / data['at']
data['Size'] = np.log(data['sale'])
data['MB'] = (data['prcc_f'] * data['cshpri'] + data['dlc'] + data['dltt'] + data['pstkl'] - data['txditc']) / data['at']
data['D_RD'] = np.where(data['xrd'].fillna(0) > 0, 1, 0)

data['RD_rank'] = pd.cut(data['xrd'] / data['sale'], bins=[-np.inf, 0, 0.25, 0.5, 0.75, np.inf], labels=[1, 2, 3, 4, 5])

data['D_Debt'] = np.where(data['Debt'] > 0.25, 1, 0)
data['D_DecIPO'] = pd.to_datetime(data['ipodate']).dt.month == 12

def winsorize(series):
    return np.clip(series, series.quantile(0.01), series.quantile(0.99))

data['Debt'] = winsorize(data['Debt'])
data['Size'] = winsorize(data['Size'])
data['MB'] = winsorize(data['MB'])

summary_stats = data[['Debt', 'Size', 'MB', 'D_RD', 'RD_rank', 'D_Debt', 'D_DecIPO']].describe(percentiles=[.25, .5, .75])
summary_stats.to_csv('summary_statistics.csv')

summary_stats_with_rd = data[data['D_RD'] == 1][['Debt', 'Size', 'MB', 'D_RD', 'RD_rank', 'D_Debt', 'D_DecIPO']].describe(percentiles=[.25, .5, .75])
summary_stats_without_rd = data[data['D_RD'] == 0][['Debt', 'Size', 'MB', 'D_RD', 'RD_rank', 'D_Debt', 'D_DecIPO']].describe(percentiles=[.25, .5, .75])
summary_stats_with_rd.to_csv('summary_statistics_with_rd.csv')
summary_stats_without_rd.to_csv('summary_statistics_without_rd.csv')



In [6]:
# Part B & C & D

def format_coef(coef, se, pval):
    stars = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.1 else ''))
    return f"{coef:.3f}{stars}\n({se:.3f})"

# Base model (Part B)
X_base = sm.add_constant(data[['Size', 'MB', 'D_RD']])
model_base = sm.OLS(data['Debt'], X_base).fit()

# Interaction model (Part C)
data['MB_D_RD'] = data['MB'] * data['D_RD']
X_interact = sm.add_constant(data[['Size', 'MB', 'D_RD', 'MB_D_RD']])
model_interact = sm.OLS(data['Debt'], X_interact).fit()

# Split samples (Part D)
rd_firms = data[data['D_RD'] == 1]
non_rd_firms = data[data['D_RD'] == 0]

X_rd = sm.add_constant(rd_firms[['Size', 'MB']])
X_non_rd = sm.add_constant(non_rd_firms[['Size', 'MB']])

model_rd = sm.OLS(rd_firms['Debt'], X_rd).fit()
model_non_rd = sm.OLS(non_rd_firms['Debt'], X_non_rd).fit()

# Full interaction model (Part D)
data['Size_D_RD'] = data['Size'] * data['D_RD']
X_full_interact = sm.add_constant(data[['Size', 'MB', 'D_RD', 'Size_D_RD', 'MB_D_RD']])
model_full_interact = sm.OLS(data['Debt'], X_full_interact).fit()

# RD rank model (Part D)
X_rank = sm.add_constant(data[['Size', 'MB', 'RD_rank']])
model_rank = sm.OLS(data['Debt'], X_rank).fit()

# Create dummy variables for RD rank categories
rd_dummies = pd.get_dummies(data['RD_rank'], prefix='RD_cat', drop_first=True)
X_nonlinear = sm.add_constant(pd.concat([data[['Size', 'MB']], rd_dummies.astype(float)], axis=1))
model_nonlinear = sm.OLS(data['Debt'], X_nonlinear).fit()

# Create combined results table
results_table = pd.DataFrame(index=[
    'Intercept', 'Size', 'MB', 'D_RD', 'MB×D_RD', 'Size×D_RD',
    'RD_rank', 'RD 2', 'RD 3', 'RD 4', 'RD 5',
    'R-squared', 'N'
])

# Add results for each model
models = {
    '(1) Base': model_base,
    '(2) Interaction': model_interact,
    '(3) RD=1': model_rd,
    '(4) RD=0': model_non_rd,
    '(5) Full Int.': model_full_interact,
    '(6) RD Rank': model_rank,
    '(7) NonLinear': model_nonlinear
}

for name, model in models.items():
    col_data = {}
    
    # Add coefficients that exist in this model
    for var in model.params.index:
        if var == 'const':
            col_data['Intercept'] = format_coef(
                model.params[var],
                model.bse[var],
                model.pvalues[var]
            )
        else:
            mapped_var = {
                'MB_D_RD': 'MB×D_RD',
                'Size_D_RD': 'Size×D_RD',
                'RD_cat_2': 'RD 2',
                'RD_cat_3': 'RD 3',
                'RD_cat_4': 'RD 4',
                'RD_cat_5': 'RD 5'
            }.get(var, var)
            
            col_data[mapped_var] = format_coef(
                model.params[var],
                model.bse[var],
                model.pvalues[var]
            )
    
    # Add R-squared and N
    col_data['R-squared'] = f"{model.rsquared:.3f}"
    col_data['N'] = f"{model.nobs}"
    
    results_table[name] = pd.Series(col_data)

print("Table 2: Regression Results")
print("=" * 120)
print(results_table.to_string(na_rep=''))

X = sm.add_constant(data[['Size', 'MB', 'D_RD']])

X_means = pd.DataFrame({
    'const': [1],
    'Size': [X['Size'].mean()],
    'MB': [X['MB'].mean()],
    'D_RD': [X['D_RD'].mean(),],
    'MB_D_RD': [X['MB'].mean() * X['D_RD'].mean()],
    'Size_D_RD': [X['Size'].mean() * X['D_RD'].mean()]
})


X_medians = pd.DataFrame({
    'const': [1],
    'Size': [X['Size'].median()],
    'MB': [X['MB'].median()],
    'D_RD': [X['D_RD'].median()],
    'MB_D_RD': [X['MB'].median() * X['D_RD'].median()],
    'Size_D_RD': [X['Size'].median() * X['D_RD'].median()]
})

# baseline model in eq1
mb_25th = X['MB'].quantile(0.25)
mb_75th = X['MB'].quantile(0.75)

predicted_means_base = float(model_base.predict(X_means.iloc[:, :4])[0])
predicted_medians_base = float(model_base.predict(X_medians.iloc[:, :4])[0])


print("\nFor baseline model:")
print("-" * 80)
print(f"Predicted Debt (means): {predicted_means_base:.3f}")
print(f"Predicted Debt (medians): {predicted_medians_base:.3f}")

# interaction model in eq2
predicted_means_interact = float(model_interact.predict(X_means.iloc[:, :5])[0])
predicted_medians_interact = float(model_interact.predict(X_medians.iloc[:, :5])[0])
effect_25th_interact = float(model_interact.params['D_RD'] + mb_25th * model_interact.params['MB_D_RD'])
effect_75th_interact = float(model_interact.params['D_RD'] + mb_75th * model_interact.params['MB_D_RD'])

print("\nFor interaction model:")
print("-" * 80)
print(f"Predicted Debt (means): {predicted_means_interact:.3f}")
print(f"Predicted Debt (medians): {predicted_medians_interact:.3f}")
print(f"Effect of MB (25th percentile): {effect_25th_interact:.3f}")
print(f"Effect of MB (75th percentile): {effect_75th_interact:.3f}")



Table 2: Regression Results
                     (1) Base     (2) Interaction           (3) RD=1            (4) RD=0       (5) Full Int.         (6) RD Rank       (7) NonLinear
Intercept   0.111***\n(0.007)   0.130***\n(0.007)  0.029***\n(0.006)   0.103***\n(0.013)   0.103***\n(0.012)   0.037***\n(0.009)   0.052***\n(0.007)
Size        0.024***\n(0.001)   0.024***\n(0.001)  0.023***\n(0.001)   0.028***\n(0.002)   0.028***\n(0.002)   0.030***\n(0.001)   0.033***\n(0.001)
MB         -0.003***\n(0.001)  -0.017***\n(0.002)     0.002\n(0.001)  -0.017***\n(0.002)  -0.017***\n(0.002)  -0.007***\n(0.001)  -0.007***\n(0.001)
D_RD       -0.077***\n(0.004)  -0.109***\n(0.006)                                         -0.074***\n(0.014)                                        
MB×D_RD                         0.019***\n(0.003)                                          0.019***\n(0.003)                                        
Size×D_RD                                                                     

In [84]:
rd_dummies
model_nonlinear.params
model_nonlinear.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Debt   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     388.2
Date:                Sat, 08 Feb 2025   Prob (F-statistic):               0.00
Time:                        16:23:20   Log-Likelihood:                 2276.0
No. Observations:               16257   AIC:                            -4538.
Df Residuals:                   16250   BIC:                            -4484.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0272      0.005      4.964      0.000       0.016       0.038
Size           0.0334      0.001     32.717      0.000       0.031       0.035
MB            -0.0067      0.001     -6.001      0.000      -0.009      -0.005
RD_cat_1       0.0250      0.005      5.435      0.000       0.016       0.034
RD_cat_2      -0.0561      0.004    -13.590      0.000      -0.064      -0.048
RD_cat_3      -0.0674      0.007    -10.171      0.000      -0.080      -0.054
RD_cat_4       0.0461      0.012      3.738      0.000       0.022       0.070
RD_cat_5       0.0797      0.006     13.244      0.000       0.068       0.091
==============================================================================
Omnibus:                     4154.138   Durbin-Watson:                   0.507
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9830.202
Skew:                           1.433   Prob(JB):                         0.00
Kurtosis:                       5.510   Cond. No.                     2.69e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.48e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [126]:
# for debug
atexog_25th


{1: 5.99144454690798, 2: 0.9232595075510989, 3: 1.0}

In [18]:
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import Logit, Probit

# Logistic regression model for eq. (1)
logit_model_1 = smf.logit('D_Debt ~ Size + MB + D_RD', data=data).fit()
print(logit_model_1.summary())

# Predict D(Debt) at means and medians
means = data[['Size', 'MB', 'D_RD']].mean()
medians = data[['Size', 'MB', 'D_RD']].median()

# Add 'const' column to means and medians
means['const'] = 1
medians['const'] = 1

# Ensure the DataFrame used for prediction has the required columns
means_df = pd.DataFrame([means])
medians_df = pd.DataFrame([medians])

pred_means = logit_model_1.predict(means_df)
pred_medians = logit_model_1.predict(medians_df)

# Marginal effects at medians
marginal_effects = logit_model_1.get_margeff(at='median').summary()

var_idx = {name: idx for idx, name in enumerate(logit_model_1.model.exog_names)}

# Create atexog for 25th percentile of MB
atexog_25th = {
    var_idx['Size']: medians['Size'],
    var_idx['MB']: data['MB'].quantile(0.25),
    var_idx['D_RD']: medians['D_RD']
}

# Create atexog for 75th percentile of MB
atexog_75th = {
    var_idx['Size']: medians['Size'],
    var_idx['MB']: data['MB'].quantile(0.75),
    var_idx['D_RD']: medians['D_RD']
}


marginal_effects_25th = logit_model_1.get_margeff(atexog=atexog_25th).summary()
marginal_effects_75th = logit_model_1.get_margeff(atexog=atexog_75th).summary()

# Logistic regression model for eq. (2) with interaction term
logit_model_2 = smf.logit('D_Debt ~ Size + MB + D_RD + MB_D_RD', data=data).fit()
print(logit_model_2.summary())

# Add interaction term to means and medians
means['MB_D_RD'] = means['MB'] * means['D_RD']
medians['MB_D_RD'] = medians['MB'] * medians['D_RD']

# Ensure the DataFrame used for prediction has the required columns for eq. (2)
means_df_2 = pd.DataFrame([means])
medians_df_2 = pd.DataFrame([medians])

# Predict D(Debt) at means and medians for eq. (2)
pred_means_2 = logit_model_2.predict(means_df_2)
pred_medians_2 = logit_model_2.predict(medians_df_2)

# Marginal effects for eq. (2) at medians
marginal_effects_2 = logit_model_2.get_margeff(at='median').summary()

def calculate_me_drd(model, mb_value):
    # Create evaluation point with medians for other variables
    X_eval = medians.copy()
    X_eval['MB'] = mb_value
    X_eval['MB_D_RD'] = mb_value  
    
    # Convert to DataFrame with one row
    X_df = pd.DataFrame([X_eval])
    
    # Get predicted probability
    p = model.predict(X_df)[0]
    
    # Get coefficients for D_RD and interaction term
    beta_drd = model.params['D_RD']
    beta_interaction = model.params['MB_D_RD']
    
    # Calculate total effect
    me = (beta_drd + beta_interaction * mb_value) * p * (1-p)
    
    return me

# Calculate for 25th and 75th percentiles of MB
mb_25th = data['MB'].quantile(0.25)
mb_75th = data['MB'].quantile(0.75)

me_25th = calculate_me_drd(logit_model_2, mb_25th)
me_75th = calculate_me_drd(logit_model_2, mb_75th)

print(f"\nMarginal Effects of D_RD:")
print(f"At MB 25th percentile ({mb_25th:.3f}): {me_25th:.4f}")
print(f"At MB 75th percentile ({mb_75th:.3f}): {me_75th:.4f}")

# Probit regression model for eq. (1)
probit_model_1 = smf.probit('D_Debt ~ Size + MB + D_RD', data=data).fit()
print(probit_model_1.summary())

# Probit regression model for eq. (2) with interaction term
probit_model_2 = smf.probit('D_Debt ~ Size + MB + D_RD + MB_D_RD', data=data).fit()
print(probit_model_2.summary())

# Custom Tobit model using censored regression
def tobit_log_likelihood(params, exog, endog, left, right):
    model = sm.OLS(endog, exog)
    llf = model.loglike(params)
    left_censored = endog <= left
    right_censored = endog >= right
    uncensored = (~left_censored & ~right_censored)
    
    llf[uncensored] = model.loglikeobs(params)
    llf[left_censored] = np.log(model.cdf(left, params))
    llf[right_censored] = np.log(1 - model.cdf(right, params))
    
    return llf.sum()

class Tobit(sm.GLM):
    def __init__(self, endog, exog, left=np.NINF, right=np.inf, **kwargs):
        self.left = left
        self.right = right
        super(Tobit, self).__init__(endog, exog, **kwargs)
        self.loglike = lambda params: tobit_log_likelihood(params, self.exog, self.endog, self.left, self.right)

# Tobit model for eq. (1) with censoring at 0 and 1
exog = sm.add_constant(data[['Size', 'MB', 'D_RD']])
endog = data['Debt']
tobit_model_1 = Tobit(endog, exog, left=0, right=1).fit()
print(tobit_model_1.summary())

def format_coef(coef, se, pval):
    stars = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.1 else ''))
    return f"{coef:.3f}{stars}\n({se:.3f})"

models = {
    '(1) Logit': logit_model_1,
    '(2) Logit+Int': logit_model_2,
    '(3) Probit': probit_model_1,
    '(4) Probit+Int': probit_model_2,
    '(5) Tobit': tobit_model_1
}

# Initialize results table with index
index = ['Intercept', 'Size', 'MB', 'D_RD', 'MB×D_RD']
table_3 = pd.DataFrame(index=index)

# Fill results
for name, model in models.items():
    col_data = {}
    
    # Map parameter names
    param_map = {
        'Intercept': 'Intercept',
        'Size': 'Size',
        'MB': 'MB',
        'D_RD': 'D_RD',
        'MB_D_RD': 'MB×D_RD'
    }
    
    # Add coefficients
    for param in model.params.index:
        mapped_name = param_map.get(param, param)
        if mapped_name in index:
            col_data[mapped_name] = format_coef(
                model.params[param],
                model.bse[param],
                model.pvalues[param]
            )
    table_3[name] = pd.Series(col_data)

print("Table 3: Limited Dependent Variable Models")
print("=" * 120)
print(table_3.to_string(na_rep=''))
table_3.to_csv('table_3.csv', index=False)

# Additional analysis
print("\nPredicted probabilities:")
print("-" * 80)
print("Logit model (eq. 1):")
print(f"Predicted probability (means): {pred_means.iloc[0]:.3f}")
print(f"Predicted probability (medians): {pred_medians.iloc[0]:.3f}")
print("\nLogit model (eq. 2):")
print(f"Predicted probability (means): {pred_means_2.iloc[0]:.3f}")
print(f"Predicted probability (medians): {pred_medians_2.iloc[0]:.3f}")

print("\nMarginal effects:")
print("-" * 80)

print("Logit model (eq. 1) at median")
print(marginal_effects)
print("\nMarginal effects of RD for firms at the 25th and 75th percentiles of MB:")
print(marginal_effects_25th)
print(marginal_effects_75th)


def calculate_marginal_effects(model, eval_point):

    p = model.predict(pd.DataFrame([eval_point]))
    
    # Get all coefficients
    betas = model.params
    
    marginal_effects = {}
    
    # For Size (no interaction)
    marginal_effects['Size'] = betas['Size'] * p * (1-p)
    
    # For MB (with interaction with D_RD)
    marginal_effects['MB'] = (betas['MB'] + betas['MB_D_RD'] * eval_point['D_RD']) * p * (1-p)
    
    # For D_RD (with interaction with MB)
    marginal_effects['D_RD'] = (betas['D_RD'] + betas['MB_D_RD'] * eval_point['MB']) * p * (1-p)
    
    return {k: float(v) for k, v in marginal_effects.items()}

marginal_effects_2 = calculate_marginal_effects(logit_model_2, medians)
var_idx['MB_D_RD'] = max(var_idx.values()) + 1
mb25th = medians.copy()
mb25th['MB'] = data['MB'].quantile(0.25)
mb25th['MB_D_RD'] = mb25th['MB'] * mb25th['D_RD']
marginal_effects_25th_2 = calculate_marginal_effects(logit_model_2, mb25th)
mb75th = medians.copy()
mb75th['MB'] = data['MB'].quantile(0.75)
mb75th['MB_D_RD'] = mb75th['MB'] * mb75th['D_RD']
marginal_effects_75th_2 = calculate_marginal_effects(logit_model_2, mb75th)

print("\nLogit model (eq. 2) at median")
print(marginal_effects_2)
print("\nMarginal effects of RD for firms at the 25th and 75th percentiles of MB:")
print(marginal_effects_25th_2)
print(marginal_effects_75th_2)

Optimization terminated successfully.
         Current function value: 0.584486
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 D_Debt   No. Observations:                16257
Model:                          Logit   Df Residuals:                    16253
Method:                           MLE   Df Model:                            3
Date:                Sun, 09 Feb 2025   Pseudo R-squ.:                 0.08471
Time:                        18:50:58   Log-Likelihood:                -9502.0
converged:                       True   LL-Null:                       -10381.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8112      0.077    -23.477      0.000      -1.962      -1.660
Size           0.2749      0.

/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_20687/1755449398.py:215: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return {k: float(v) for k, v in marginal_effects.items()}
/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_20687/1755449398.py:215: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return {k: float(v) for k, v in marginal_effects.items()}
/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_20687/1755449398.py:215: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return {k: float(v) for k, v in marginal_effects.items()}


In [112]:

# examine intermedate results
print(atexog_25th)
print(atexog_75th)

Size       5.991445
MB         0.923260
D_RD       1.000000
MB_D_RD    0.923260
dtype: float64
Size       5.991445
MB         2.343715
D_RD       1.000000
MB_D_RD    2.343715
dtype: float64


In [6]:
# discrete examine some outputs
pred_means
marginal_effects

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 D_Debt
Method:                          dydx
At:                            median
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Size           0.0561      0.002     25.189      0.000       0.052       0.060
MB            -0.0226      0.003     -8.007      0.000      -0.028      -0.017
D_RD          -0.1220      0.007    -18.479      0.000      -0.135      -0.109
==============================================================================
"""

In [7]:
# discrete examine some outputs
pred_medians
marginal_effects_2

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 D_Debt
Method:                          dydx
At:                            median
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Size           0.0527      0.002     24.665      0.000       0.049       0.057
MB            -0.0488      0.004    -11.310      0.000      -0.057      -0.040
D_RD          -0.1812      0.009    -20.762      0.000      -0.198      -0.164
MB_D_RD        0.0423      0.005      8.391      0.000       0.032       0.052
==============================================================================
"""

In [35]:
# Part F IV Analysis
from linearmodels.iv import IV2SLS

# First Stage: Regress MB on the instrument (D_DecIPO) and other exogenous variables (Size, D_RD)
first_stage = smf.ols('MB ~ Size + D_RD + D_DecIPO', data=data).fit()
data['MB_hat'] = first_stage.fittedvalues

# Second Stage: Regress Debt on the predicted values of MB (MB_hat) and other exogenous variables (Size, D_RD)
dependent = data['Debt']
endog = data[['MB']]
exog = sm.add_constant(data[['Size', 'D_RD']])  # Exogenous variables
instruments = data[['D_DecIPO']]


iv2sls_model = IV2SLS(dependent, exog, endog, instruments).fit(cov_type='unadjusted')

# OLS estimation of eq. (1) for comparison
ols_model = smf.ols('Debt ~ Size + MB + D_RD', data=data).fit()

def format_params(model, variables):
    params = model.params
    pvalues = model.pvalues

    if hasattr(model, 'bse'):  # For statsmodels results
        bse = model.bse
    else:  # For linearmodels results
        bse = model.std_errors

    formatted_params = {
        var: (
            f'{params[var]:.3f}'
            f'{"***" if pvalues[var] < 0.01 else ("**" if pvalues[var] < 0.05 else ("*" if pvalues[var] < 0.1 else ""))}\n'
            f'({bse[var]:.3f})'
        ) if var in params else ''
        for var in variables
    }
    return formatted_params

variables_first_stage = ['Intercept', 'Size', 'D_RD', 'D_DecIPO[T.True]']
variables_second_stage = ['const', 'Size', 'MB', 'D_RD']

params_first_stage = format_params(first_stage, variables_first_stage)
params_second_stage = format_params(iv2sls_model, variables_second_stage)

rows = []

# Add first stage results
for var in variables_first_stage:
    rows.append({
        'Variables': var,
        'First Stage (MB)': params_first_stage.get(var, ''),
        'Second Stage (Debt)': ''
    })

# Add separator
rows.append({
    'Variables': '',
    'First Stage (MB)': '',
    'Second Stage (Debt)': ''
})

# Add second stage results
for var in variables_second_stage:
    rows.append({
        'Variables': var,
        'First Stage (MB)': '',
        'Second Stage (Debt)': params_second_stage.get(var, '')
    })

# Create Table 4 from rows
table_4 = pd.DataFrame(rows)

# Add statistics
stats_rows = [
    {
        'Variables': 'R-squared',
        'First Stage (MB)': f'{first_stage.rsquared:.3f}',
        'Second Stage (Debt)': f'{iv2sls_model.rsquared:.3f}'
    },
    {
        'Variables': 'N',
        'First Stage (MB)': str(len(data)),
        'Second Stage (Debt)': str(len(data))
    }
]

# Append statistics to the table
table_4 = pd.concat([table_4, pd.DataFrame(stats_rows)], ignore_index=True)

# Save Table 4 to CSV
table_4.to_csv('table4_iv_analysis.csv', index=False)

# Print results
print("Table 4: Two-Stage Least Squares (2SLS) Analysis")
print("=" * 80)
print(table_4.to_string(index=False))

Table 4: Two-Stage Least Squares (2SLS) Analysis
       Variables   First Stage (MB) Second Stage (Debt)
       Intercept  2.284***\n(0.044)                    
            Size -0.114***\n(0.006)                    
            D_RD  0.525***\n(0.025)                    
D_DecIPO[T.True] -0.223***\n(0.041)                    
                                                       
           const                      0.250***\n(0.065)
            Size                      0.017***\n(0.003)
              MB                      -0.065**\n(0.029)
            D_RD                     -0.044***\n(0.016)
       R-squared              0.071              -0.061
               N              16257               16257
     F-statistic             412.16                    


In [ ]:
# first stage shows D_DecIPO is significant in explaining the variation in MB.
# second stage shows that the predicted values of MB (MB_hat) are significant in explaining the variation in Debt.
# This suggests that the instrument (D_DecIPO) is valid and that the effect of MB on Debt is significant.

In [34]:
first_stage.summary()
iv2sls_model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                   Debt   R-squared:                     -0.0614
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0616
No. Observations:               16257   F-statistic:                    1612.1
Date:                Fri, Feb 07 2025   P-value (F-stat)                0.0000
Time:                        15:27:53   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.2503     0.0651     3.8454     0.0001      0.1227      0.3780
Size           0.0167     0.0034     4.9358     0.0000      0.0101      0.0234
D_RD          -0.0445     0.0157    -2.8401     0.0045     -0.0752     -0.0138
MB            -0.0648     0.0286    -2.2631     0.0236     -0.1209     -0.0087
==============================================================================

Endogenous: MB
Instruments: D_DecIPO
Unadjusted Covariance (Homoskedastic)
Debiased: False
"""

In [ ]:
# place holder for the results

In [ ]:
# Part G Panel Data Estimations

from linearmodels.panel import PanelOLS, RandomEffects

# Set the index for panel data
data_panel = data.set_index(['gvkey', 'fyear'])

# Firm-Level Fixed Effects (FE)
fe_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + EntityEffects', data=data_panel, drop_absorbed=True).fit()
print(fe_model.summary)

# Firm-Level Random Effects (RE)
re_model = RandomEffects.from_formula('Debt ~ 1 + Size + MB + D_RD', data=data_panel).fit()
print(re_model.summary)

# Firm-Level Fixed Effects with Year Fixed Effects
fe_year_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + EntityEffects + TimeEffects', data=data_panel, drop_absorbed=True).fit()
print(fe_year_model.summary)

# Firm-Level Random Effects with Year Fixed Effects
re_year_model = RandomEffects.from_formula('Debt ~ 1 + Size + MB + D_RD + TimeEffects', data=data_panel).fit()
print(re_year_model.summary)

# Augmented Model with D(DecIPO) as an additional explanatory variable
fe_augmented_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + D_DecIPO + EntityEffects', data=data_panel, drop_absorbed=True).fit()
print(fe_augmented_model.summary)

re_augmented_model = RandomEffects.from_formula('Debt ~ 1 + Size + MB + D_RD + D_DecIPO', data=data_panel).fit()
print(re_augmented_model.summary)

# OLS estimation of eq. (1) for comparison
ols_model = smf.ols('Debt ~ Size + MB + D_RD', data=data_panel.reset_index()).fit()

# Function to format the parameters
def format_params(model, variables):
    params = model.params
    bse = model.bse if hasattr(model, 'bse') else model.std_errors
    pvalues = model.pvalues

    formatted_params = {
    var: (
        f'{params[var]:.3f}'
        f'{"***" if pvalues[var] < 0.01 else ("**" if pvalues[var] < 0.05 else ("*" if pvalues[var] < 0.1 else ""))}\n'
        f'({bse[var]:.3f})'
    ) if var in params else ''
    for var in variables
}
    return formatted_params

variables = ['Intercept', 'Size', 'MB', 'D_RD', 'D_DecIPO']

params_ols = format_params(ols_model, variables)
params_fe = format_params(fe_model, variables)
params_re = format_params(re_model, variables)
params_fe_year = format_params(fe_year_model, variables)
params_re_year = format_params(re_year_model, variables)
params_fe_augmented = format_params(fe_augmented_model, variables)
params_re_augmented = format_params(re_augmented_model, variables)

table_5 = pd.DataFrame({
    'Variables': variables,
    'OLS': [params_ols[var] for var in variables],
    'FE': [params_fe[var] for var in variables],
    'RE': [params_re[var] for var in variables],
    'FE Year': [params_fe_year[var] for var in variables],
    'RE Year': [params_re_year[var] for var in variables],
    'FE Augmented': [params_fe_augmented[var] for var in variables],
    'RE Augmented': [params_re_augmented[var] for var in variables]
})

# Save the results to a CSV file
table_5.to_csv('panel_data_estimations_table_5.csv', index=False)

# Print the results
print("Table 5: Panel Data Estimations (Fixed Effects and Random Effects)")
print(table_5)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Debt   R-squared:                        0.0176
Estimator:                   PanelOLS   R-squared (Between):              0.1278
No. Observations:               16257   R-squared (Within):               0.0176
Date:                Sat, Feb 08 2025   R-squared (Overall):              0.0949
Time:                        13:37:46   Log-likelihood                 1.365e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      82.393
Entities:                        2467   P-value                           0.0000
Avg Obs:                       6.5898   Distribution:                 F(3,13787)
Min Obs:                       1.0000                                           
Max Obs:                       12.000   F-statistic (robust):             82.393
                            

/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_49227/1376311673.py:23: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

D_DecIPO

  fe_augmented_model = PanelOLS.from_formula('Debt ~ 1 + Size + MB + D_RD + D_DecIPO + EntityEffects', data=data_panel, drop_absorbed=True).fit()


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Debt   R-squared:                        0.0176
Estimator:                   PanelOLS   R-squared (Between):              0.1278
No. Observations:               16257   R-squared (Within):               0.0176
Date:                Sat, Feb 08 2025   R-squared (Overall):              0.0949
Time:                        13:37:46   Log-likelihood                 1.365e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      82.393
Entities:                        2467   P-value                           0.0000
Avg Obs:                       6.5898   Distribution:                 F(3,13787)
Min Obs:                       1.0000                                           
Max Obs:                       12.000   F-statistic (robust):             82.393
                            

In [9]:
data

sic state    ipodate  lpermno                conm  exchg  \
gvkey  fyear                                                               
001004 2005.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2006.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2007.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2008.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
       2009.0  5080    IL 1972-04-24  54594.0            AAR CORP   11.0   
...             ...   ...        ...      ...                 ...    ...   
287252 2016.0  7370    FL 2007-11-12  14075.0  MIX TELEMATICS LTD   11.0   
311524 2013.0  2860    PA 2013-04-18  13861.0        TAMINCO CORP   11.0   
317264 2014.0  4412    CT 2014-05-08  14642.0      DORIAN LPG LTD   11.0   
       2015.0  4412    CT 2014-05-08  14642.0      DORIAN LPG LTD   11.0   
       2016.0  4412    CT 2014-05-08  14642.0      DORIAN LPG LTD   11.0   

                   act        at      dlc     dltt  ...  prcc_f      Debt  \
gvkey  fyear                                        ...                     
001004 2005.0  624.454   978.819    2.289  318.576  ...   24.08  0.327808   
       2006.0  645.721  1067.633   74.245  253.611  ...   32.50  0.307087   
       2007.0  783.431  1362.010   22.994  507.918  ...   19.28  0.389800   
       2008.0  851.312  1377.511   63.600  392.984  ...   14.70  0.331456   
       2009.0  863.429  1501.042  100.833  336.191  ...   19.70  0.291147   
...                ...       ...      ...      ...  ...     ...       ...   
287252 2016.0   52.339   142.052    1.449    0.000  ...    6.30  0.010200   
311524 2013.0  311.000  1856.000    7.000  908.000  ...   20.21  0.492996   
317264 2014.0  233.211  1099.101   15.678  184.666  ...   13.03  0.182280   
       2015.0  105.559  1865.926   66.265  770.103  ...    9.40  0.448232   
       2016.0   63.971  1746.235   65.979  683.985  ...   10.53  0.429475   

                   Size        MB  D_RD  RD_rank  D_Debt  D_DecIPO   MB_D_RD  \
gvkey  fyear                                                                   
001004 2005.0  6.799372  1.126777     0        1       1     False  0.000000   
       2006.0  6.967126  1.377231     0        1       1     False  0.000000   
       2007.0  7.233397  0.895736     0        1       1     False  0.000000   
       2008.0  7.261208  0.708371     0        1       1     False  0.000000   
       2009.0  7.209452  0.754059     0        1       1     False  0.000000   
...                 ...       ...   ...      ...     ...       ...       ...   
287252 2016.0  4.742643  1.074673     1        2       0     False  1.074673   
311524 2013.0  7.090077  1.077520     1        2       1     False  1.077520   
317264 2014.0  4.646533  0.858817     0        1       0     False  0.000000   
       2015.0  5.673849  0.733659     0        1       1     False  0.000000   
       2016.0  5.134963  0.755577     0        1       1     False  0.000000   

                 MB_hat  
gvkey  fyear             
001004 2005.0  1.511017  
       2006.0  1.491941  
       2007.0  1.461663  
       2008.0  1.458501  
       2009.0  1.464386  
...                 ...  
287252 2016.0  2.269484  
311524 2013.0  2.002556  
317264 2014.0  1.755816  
       2015.0  1.639000  
       2016.0  1.700277  

[16257 rows x 31 columns]

In [50]:
# Part H: Difference-in-Differences Analysis

# Create the D(Post) and D(Treated) indicators
data['D_Post'] = np.where(data['fyear'] > 2010, 1, 0)
data['D_Treated'] = np.where(data['state'].isin(['NY', 'CA']), 1, 0)
data['D_Post_x_Treated'] = data['D_Post'] * data['D_Treated']

years = sorted(data['fyear'].unique())
base_year = 2010
for year in years:
    if year != base_year:  # Skip the reference year
        data[f'D_Treated_x_year_{year}'] = data['D_Treated'] * (data['fyear'] == year).astype(int)


# Base controls from equation (1)
controls = ['Size', 'MB', 'D_RD']

def run_dd_regressions(data):
    """Run all DD specifications and return formatted results"""
    
    # 1. Baseline DD without FE
    X = sm.add_constant(data[['D_Post', 'D_Treated', 'D_Post_x_Treated'] + controls])
    y = data['Debt']
    baseline = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data['gvkey']})
    
    # 2. DD with firm FE
    data_panel = data.set_index(['gvkey', 'fyear'])
    exog_vars = ['D_Post', 'D_Post_x_Treated'] + controls
    mod_firm_fe = PanelOLS(data_panel['Debt'], 
                          data_panel[exog_vars],
                          entity_effects=True)
    firm_fe = mod_firm_fe.fit(cov_type='clustered', cluster_entity=True)
    
    # 3. DD with firm and year FE
    exog_vars_twoway = ['D_Post_x_Treated'] + controls
    mod_twoway_fe = PanelOLS(data_panel['Debt'],
                            data_panel[exog_vars_twoway],
                            entity_effects=True,
                            time_effects=True,
                            drop_absorbed=True)
    twoway_fe = mod_twoway_fe.fit(cov_type='clustered', cluster_entity=True)
    
    # 4. Dynamic DD with firm FE
    dynamic_vars = [col for col in data.columns if 'D_Treated_x_year' in col] + controls
    mod_dynamic = PanelOLS(data_panel['Debt'],
                          data_panel[dynamic_vars],
                          entity_effects=True,
                          drop_absorbed=True)
    dynamic = mod_dynamic.fit(cov_type='clustered', cluster_entity=True)
    
    return baseline, firm_fe, twoway_fe, dynamic

def format_coefficient(coef, se, pval):
    """Format coefficient with stars and standard error"""
    stars = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.1 else ''))
    return f'{coef:.3f}{stars}\n({se:.3f})'

def create_table_5(baseline, firm_fe, twoway_fe, dynamic):
    """Create Table 5 with all regression results"""
    
    rows = []
    
    # Variables of interest for each specification
    main_vars = ['D_Post', 'D_Treated', 'D_Post_x_Treated']
    year_vars = [col for col in data.columns if 'D_Treated_x_year' in col]
    
    # Add main DD coefficients
    for var in main_vars:
        row = {'Variables': var}
        
        # Baseline
        if var in baseline.params.index:
            row['(1) Baseline'] = format_coefficient(
                baseline.params[var],
                baseline.bse[var],
                baseline.pvalues[var]
            )
        else:
            row['(1) Baseline'] = ''
            
        # Firm FE
        if var in firm_fe.params.index:
            row['(2) Firm FE'] = format_coefficient(
                firm_fe.params[var],
                firm_fe.std_errors[var],
                firm_fe.pvalues[var]
            )
        else:
            row['(2) Firm FE'] = ''
            
        # Firm and Year FE
        if var in twoway_fe.params.index:
            row['(3) Firm+Year FE'] = format_coefficient(
                twoway_fe.params[var],
                twoway_fe.std_errors[var],
                twoway_fe.pvalues[var]
            )
        else:
            row['(3) Firm+Year FE'] = ''
            
        row['(4) Dynamic'] = ''
        rows.append(row)
    
    # Add separator
    rows.append({col: '' for col in rows[0].keys()})
    
    # Add dynamic effects
    for var in sorted(year_vars):
        row = {'Variables': var}
        row['(1) Baseline'] = ''
        row['(2) Firm FE'] = ''
        row['(3) Firm+Year FE'] = ''
        
        if var in dynamic.params.index:
            row['(4) Dynamic'] = format_coefficient(
                dynamic.params[var],
                dynamic.std_errors[var],
                dynamic.pvalues[var]
            )
        rows.append(row)
    
    # Add model statistics
    stats_rows = [
        {
            'Variables': 'R-squared',
            '(1) Baseline': f'{baseline.rsquared:.3f}',
            '(2) Firm FE': f'{firm_fe.rsquared:.3f}',
            '(3) Firm+Year FE': f'{twoway_fe.rsquared:.3f}',
            '(4) Dynamic': f'{dynamic.rsquared:.3f}'
        },
        {
            'Variables': 'N',
            '(1) Baseline': str(baseline.nobs),
            '(2) Firm FE': str(firm_fe.nobs),
            '(3) Firm+Year FE': str(twoway_fe.nobs),
            '(4) Dynamic': str(dynamic.nobs)
        }
    ]
    
    # Create and format table
    table_5 = pd.DataFrame(rows + stats_rows)
    
    return table_5

# Run all regressions
baseline, firm_fe, twoway_fe, dynamic = run_dd_regressions(data)

# Create and save Table 5
table_5 = create_table_5(baseline, firm_fe, twoway_fe, dynamic)
table_5.to_csv('table5_dd_analysis.csv', index=False)

# Print results with discussion
print("Table 5: Difference-in-Differences Analysis")
print("=" * 80)
print(table_5.to_string(index=False))
print("\nDiscussion:")
print("-" * 80)

# Basic DD effect
dd_effect = baseline.params['D_Post_x_Treated']
dd_pval = baseline.pvalues['D_Post_x_Treated']

print("\n1. Baseline DD Results:")
print(f"- The DD coefficient is {dd_effect:.3f} (p-value: {dd_pval:.3f})")
print("- This suggests that the shock " + 
      ("increased" if dd_effect > 0 else "decreased") + 
      " debt levels for treated firms by " +
      f"{abs(dd_effect):.3f} units")

# Compare with FE results
fe_effect = firm_fe.params['D_Post_x_Treated']
print("\n2. Firm Fixed Effects Results:")
print(f"- The DD coefficient with firm FE is {fe_effect:.3f}")
print("- The " + ("larger" if abs(fe_effect) > abs(dd_effect) else "smaller") +
      " magnitude suggests that firm-specific factors " +
      ("amplify" if abs(fe_effect) > abs(dd_effect) else "attenuate") +
      " the treatment effect")

# Discuss parallel trends
dynamic_coefs = {var: dynamic.params[var] 
                for var in dynamic.params.index 
                if 'D_Treated_x_year' in var}
pre_trend = all(abs(v) < 0.1 for k, v in dynamic_coefs.items() 
               if int(float(k.split('_')[-1])) < 2010)

def analyze_parallel_trends(data):
    # Calculate mean Debt by year for treated and control groups
    trends = data.groupby(['fyear', 'D_Treated'])['Debt'].mean().unstack()
    trends.columns = ['Control', 'Treated']
    
    # Calculate the difference between groups by year
    trends['Difference'] = trends['Treated'] - trends['Control']
    
    # Print analysis
    print("\n3. Parallel Trends Analysis:")
    print("\nMean Debt Levels:")
    print("-" * 50)
    print("Pre-treatment period (2005-2009):")
    pre_trends = trends.loc[trends.index < 2010]
    print(pre_trends)
    
    # Calculate year-over-year changes for each group
    pre_trends_diff = pre_trends.diff()
    print("\nYear-over-year changes:")
    print(pre_trends_diff)
    
    # Test if year-over-year changes are similar between groups
    control_changes = pre_trends_diff['Control'].mean()
    treated_changes = pre_trends_diff['Treated'].mean()
    print(f"\nAverage annual change pre-2010:")
    print(f"Control group: {control_changes:.3f}")
    print(f"Treated group: {treated_changes:.3f}")
    
    return trends

# Run the analysis
trends = analyze_parallel_trends(data)

# Visualize parallel trends
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(trends.index, trends['Control'], label='Control', marker='o')
plt.plot(trends.index, trends['Treated'], label='Treated', marker='o')
plt.axvline(x=2010, color='r', linestyle='--', label='Treatment Year')
plt.xlabel('Year')
plt.ylabel('Mean Debt')
plt.title('Trends in Debt: Treated vs Control Groups')
plt.legend()
plt.grid(True)
plt.savefig('parallel_trends.png')
plt.close()

/var/folders/53/z0kd2hl56_1g6ntqlk8pfcgm0000gn/T/ipykernel_49227/3741234334.py:49: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

D_Treated_x_year_2016.0

  dynamic = mod_dynamic.fit(cov_type='clustered', cluster_entity=True)


Table 5: Difference-in-Differences Analysis
              Variables       (1) Baseline      (2) Firm FE (3) Firm+Year FE        (4) Dynamic
                 D_Post  0.024***\n(0.005) 0.011**\n(0.005)                                    
              D_Treated -0.046***\n(0.011)                                                     
       D_Post_x_Treated     0.003\n(0.010)   0.014\n(0.010)   0.014\n(0.010)                   
                                                                                               
D_Treated_x_year_2005.0                                                      -0.071***\n(0.015)
D_Treated_x_year_2006.0                                                      -0.064***\n(0.014)
D_Treated_x_year_2007.0                                                      -0.051***\n(0.014)
D_Treated_x_year_2008.0                                                      -0.041***\n(0.014)
D_Treated_x_year_2009.0                                                      -0.052***\n(0.0

Matplotlib is building the font cache; this may take a moment.


In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Filter data for 2010 only
data_2010 = data[data['fyear'] == 2010].copy()

def run_base_regression(data):
    """Run basic OLS with D_Treated"""
    X = sm.add_constant(data[['D_Treated', 'Size', 'MB', 'D_RD']])
    y = data['Debt']
    model = sm.OLS(y, X)
    results = model.fit(cov_type='HC0')  # Using robust standard errors
    return results

def estimate_propensity_scores(data):  # NEW FUNCTION
    """Estimate propensity scores using logistic regression"""
    features = ['Size', 'MB', 'D_RD']
    X = sm.add_constant(data[features])
    y = data['D_Treated']
    
    logit = sm.Logit(y, X)
    ps_model = logit.fit(disp=0)
    ps_scores = ps_model.predict(X)
    return ps_scores

def perform_psm(data, n_neighbors, caliper=0.01):
    # NEW: First estimate propensity scores
    ps_scores = estimate_propensity_scores(data)
    data['ps_score'] = ps_scores

    treated = data[data['D_Treated'] == 1].copy()
    control = data[data['D_Treated'] == 0].copy()

    print(f"\nNumber of firms before matching:")
    print(f"Treated: {len(treated)}")
    print(f"Control: {len(control)}")
    
    matched_pairs = []
    used_control = set()
    
    for t_idx, t_firm in treated.iterrows():
        # Calculate distance to all control firms
        distances = pd.Series(np.abs(control['ps_score'] - t_firm['ps_score']))
        
        # Find eligible matches within caliper
        eligible = distances[distances <= caliper].sort_values()
        
        # Remove already used controls
        eligible = eligible[~eligible.index.isin(used_control)]
        
        # If we found enough matches
        if len(eligible) >= n_neighbors:
            matches = eligible.head(n_neighbors)
            matched_pairs.extend([(t_idx, c_idx) for c_idx in matches.index])
            used_control.update(matches.index)
    
    if len(matched_pairs) == 0:
        print("No matches found!")
        return None
    
    # Create matched sample
    treated_indices = [pair[0] for pair in matched_pairs]
    control_indices = [pair[1] for pair in matched_pairs]
    all_indices = list(set(treated_indices + control_indices))
    matched_sample = data.loc[all_indices].copy()
    
    print(f"\nMatching results:")
    print(f"Matched treated firms: {len(set(treated_indices))}")
    print(f"Matched control firms: {len(set(control_indices))}")
    print(f"Total sample size: {len(matched_sample)}")
    
    return matched_sample

def create_balance_table(full_sample, matched_sample):
    """Create balance table for comparison"""
    variables = ['Size', 'MB', 'D_RD', 'Debt']
    
    stats = []
    for var in variables:
        # Full sample statistics
        treated_full = full_sample[full_sample['D_Treated'] == 1][var]
        control_full = full_sample[full_sample['D_Treated'] == 0][var]
        
        # Matched sample statistics
        treated_matched = matched_sample[matched_sample['D_Treated'] == 1][var]
        control_matched = matched_sample[matched_sample['D_Treated'] == 0][var]
        
        # Calculate differences and t-tests
        diff_full = treated_full.mean() - control_full.mean()
        diff_matched = treated_matched.mean() - control_matched.mean()
        
        stats.append({
            'Variable': var,
            'Treated_Full': f"{treated_full.mean():.3f}",
            'Control_Full': f"{control_full.mean():.3f}",
            'Diff_Full': f"{diff_full:.3f}",
            'Treated_Matched': f"{treated_matched.mean():.3f}",
            'Control_Matched': f"{control_matched.mean():.3f}",
            'Diff_Matched': f"{diff_matched:.3f}"
        })
    
    return pd.DataFrame(stats)

def format_coefficient(coef, se, pval):
    """Format coefficient with stars and standard error"""
    stars = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.1 else ''))
    return f'{coef:.3f}{stars}\n({se:.3f})'

# Run baseline regression
baseline_results = run_base_regression(data_2010)

# Perform PSM with 5 neighbors
matched_sample_5 = perform_psm(data_2010, n_neighbors=5, caliper=0.01)
psm5_results = run_base_regression(matched_sample_5)

# Perform PSM with 2 neighbors
matched_sample_2 = perform_psm(data_2010, n_neighbors=2, caliper=0.01)
psm2_results = run_base_regression(matched_sample_2)

# Create Table 6
variables = ['const', 'D_Treated', 'Size', 'MB', 'D_RD']
table_rows = []

for var in variables:
    row = {'Variables': var}
    
    # Baseline results
    row['(1) Full Sample'] = format_coefficient(
        baseline_results.params[var],
        baseline_results.bse[var],
        baseline_results.pvalues[var]
    )
    
    # PSM (5) results
    row['(2) PSM (n=5)'] = format_coefficient(
        psm5_results.params[var],
        psm5_results.bse[var],
        psm5_results.pvalues[var]
    )
    
    # PSM (2) results
    row['(3) PSM (n=2)'] = format_coefficient(
        psm2_results.params[var],
        psm2_results.bse[var],
        psm2_results.pvalues[var]
    )
    
    table_rows.append(row)

# Add model statistics
stats_rows = [
    {
        'Variables': 'R-squared',
        '(1) Full Sample': f'{baseline_results.rsquared:.3f}',
        '(2) PSM (n=5)': f'{psm5_results.rsquared:.3f}',
        '(3) PSM (n=2)': f'{psm2_results.rsquared:.3f}'
    },
    {
        'Variables': 'N',
        '(1) Full Sample': str(baseline_results.nobs),
        '(2) PSM (n=5)': str(psm5_results.nobs),
        '(3) PSM (n=2)': str(psm2_results.nobs)
    }
]

# Create and save Table 6
table_6 = pd.DataFrame(table_rows + stats_rows)
table_6.to_csv('table6_matching.csv', index=False)

# Create balance table
balance_table_5 = create_balance_table(data_2010, matched_sample_5)
balance_table_2 = create_balance_table(data_2010, matched_sample_2)

# Print results
print("Table 6: Cross-sectional Analysis with Matching (Year 2010)")
print("=" * 80)
print(table_6.to_string(index=False))

print("\nPanel A: Balance Table (5-neighbor matching)")
print("=" * 80)
print(balance_table_5.to_string(index=False))

print("\nPanel B: Balance Table (2-neighbor matching)")
print("=" * 80)
print(balance_table_2.to_string(index=False))

print("\nDiscussion:")
print("-" * 80)
treated_effect = baseline_results.params['D_Treated']
print(f"\n1. Baseline Results:")
print(f"- Treatment effect in full sample: {treated_effect:.3f}")
print(f"- P-value: {baseline_results.pvalues['D_Treated']:.3f}")

print("\n2. Matching Analysis:")
print("- Number of treated firms:", sum(data_2010['D_Treated'] == 1))
print("- Number of matched pairs (n=5):", len(matched_sample_5) // 6)  # divide by 6 (1 treated + 5 controls)
print("- Number of matched pairs (n=2):", len(matched_sample_2) // 3)  # divide by 3 (1 treated + 2 controls)

print("\n3. Treatment Effects After Matching:")
print(f"- PSM (n=5): {psm5_results.params['D_Treated']:.3f}")
print(f"- PSM (n=2): {psm2_results.params['D_Treated']:.3f}")


Number of firms before matching:
Treated: 343
Control: 960

Matching results:
Matched treated firms: 184
Matched control firms: 920
Total sample size: 1104

Number of firms before matching:
Treated: 343
Control: 960

Matching results:
Matched treated firms: 324
Matched control firms: 648
Total sample size: 972
Table 6: Cross-sectional Analysis with Matching (Year 2010)
Variables    (1) Full Sample      (2) PSM (n=5)      (3) PSM (n=2)
    const  0.120***\n(0.022)  0.095***\n(0.023)  0.125***\n(0.026)
D_Treated -0.049***\n(0.012)  -0.039**\n(0.016) -0.043***\n(0.013)
     Size  0.021***\n(0.003)  0.025***\n(0.003)  0.019***\n(0.003)
       MB  -0.007**\n(0.004)   -0.009*\n(0.005)    -0.005\n(0.004)
     D_RD -0.071***\n(0.012) -0.067***\n(0.013) -0.074***\n(0.015)
R-squared              0.122              0.111              0.097
        N             1303.0             1104.0              972.0

Panel A: Balance Table (5-neighbor matching)
Variable Treated_Full Control_Full Diff_Full 

In [59]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# 1. Focus on 2010 only
data_2010 = data[data['fyear'] == 2010].copy()

# Add debug prints to see data
print("Data Preview:")
print(data_2010[['D_Treated', 'Size', 'MB', 'D_RD', 'Debt']].head())
print("\nData Summary:")
print(data_2010[['D_Treated', 'Size', 'MB', 'D_RD', 'Debt']].describe())

def estimate_ps(data):
    """Estimate propensity scores based on Size, MB, and D_RD"""
    X = sm.add_constant(data[['Size', 'MB', 'D_RD']])
    y = data['D_Treated']
    logit = sm.Logit(y, X)
    ps_model = logit.fit(disp=0)
    
    # Add debug prints
    print("\nPropensity Score Model Summary:")
    print(ps_model.summary().tables[1])
    
    ps_scores = ps_model.predict(X)
    
    # Print PS score distribution
    print("\nPropensity Score Distribution:")
    print(pd.Series(ps_scores).describe())
    
    return ps_scores

def match_firms(data, n_neighbors=5, caliper=0.1):
    """Perform nearest-neighbor matching with specified caliper"""
    # Calculate propensity scores
    ps_scores = estimate_ps(data)
    data['ps_score'] = ps_scores
    
    # Print treated vs control distributions
    print("\nPropensity Score by Group:")
    print("Treated:")
    print(data[data['D_Treated']==1]['ps_score'].describe())
    print("\nControl:")
    print(data[data['D_Treated']==0]['ps_score'].describe())
    
    treated = data[data['D_Treated'] == 1].copy()
    control = data[data['D_Treated'] == 0].copy()
    
    print(f"\nNumber of firms before matching:")
    print(f"Treated: {len(treated)}")
    print(f"Control: {len(control)}")
    
    matched_pairs = []
    used_control = set()
    
    # For each treated firm
    for t_idx, t_firm in treated.iterrows():
        # Calculate distance to all control firms
        distances = pd.Series(np.abs(control['ps_score'] - t_firm['ps_score']))
        
        # Find eligible matches within caliper
        eligible = distances[distances <= caliper].sort_values()
        
        # Debug print for first few treated firms
        if len(matched_pairs) < 3:  # Only print first 3 for brevity
            print(f"\nTreated firm {len(matched_pairs)+1}:")
            print(f"PS score: {t_firm['ps_score']:.4f}")
            print(f"Number of eligible matches within caliper: {len(eligible)}")
            if len(eligible) > 0:
                print(f"Closest match distances: {eligible.head().values}")
        
        # Remove already used controls
        eligible = eligible[~eligible.index.isin(used_control)]
        
        # If we found enough matches
        if len(eligible) >= n_neighbors:
            matches = eligible.head(n_neighbors)
            matched_pairs.extend([(t_idx, c_idx) for c_idx in matches.index])
            used_control.update(matches.index)
    
    print(f"\nNumber of matched pairs found: {len(matched_pairs)//n_neighbors}")
    
    if not matched_pairs:
        print("No matches found!")
        return None
    
    all_indices = [idx for pair in matched_pairs for idx in [pair[0], pair[1]]]
    matched_sample = data.loc[all_indices].copy()
    
    print(f"\nFinal matched sample size: {len(matched_sample)}")
    
    return matched_sample

# Run matching with debug output
print("\nRunning 5-neighbor matching:")
matched_sample_5 = match_firms(data_2010, n_neighbors=5, caliper=0.1)

print("\nRunning 2-neighbor matching:")
matched_sample_2 = match_firms(data_2010, n_neighbors=2, caliper=0.1)

Data Preview:
     D_Treated      Size        MB  D_RD      Debt
25           0  7.481996  0.796925     0  0.260533
69           0  7.053240  1.259019     1  0.441573
107          0  7.410454  1.092948     1  0.000000
157          0  9.107865  1.582094     1  0.305666
220          0  7.923530  2.405235     1  0.092550

Data Summary:
         D_Treated         Size           MB         D_RD         Debt
count  1303.000000  1303.000000  1303.000000  1303.000000  1303.000000
mean      0.263239     5.826620     1.872372     0.574827     0.175578
std       0.440560     1.970321     1.501719     0.494559     0.209911
min       0.000000    -0.977569     0.334524     0.000000     0.000000
25%       0.000000     4.692863     0.916885     0.000000     0.000059
50%       0.000000     6.011760     1.384973     1.000000     0.100649
75%       1.000000     7.169910     2.267260     1.000000     0.282905
max       1.000000    10.067340     8.824585     1.000000     1.016185

Running 5-neighbor matchi